In [1]:
import socket
import struct
import threading
import time
import sys

class listenThread (threading.Thread):
    def __init__(self, threadID, name, socket1, socket2, freq, mcast_grp1, mcast_port1, mcast_grp2, mcast_port2):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.sock1 = socket1 # Recieve
        self.sock2 = socket2 # Send
        self.freq = freq
        self.mcast_grp1 = mcast_grp1
        self.mcast_port1 = mcast_port1
        self.mcast_grp2 = mcast_grp2
        self.mcast_port2 = mcast_port2
        
    def run(self):
        print 'Thread {} "{}" Started'.format(self.threadID, self.name)
        while(1):
            time.sleep(self.freq)
            try:
                message, address = sock1.recvfrom(10240)
                if 'HeartBeat' in message:
                    # print message received
                    print '{}, {}, Message: "{}", Sender: {}'.format(time.ctime(time.time()), 'Receive', message, address)
                    
                    # send response
                    response = "Slave 2 Alive"
                    self.sock2.sendto(response, (self.mcast_grp2, self.mcast_port2))
                    print '{}, {}, Message: "{}"'.format(time.ctime(time.time()), 'Transmit', response)

            except:
                None
                #print "Unexpected error:", sys.exc_info()[0]

# Multicast Socket setup - Receive 
MCAST_GRP1 = '224.1.1.1'
MCAST_PORT1 = 5008
sock1 = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
sock1.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
sock1.bind(('', MCAST_PORT1))  # use MCAST_GRP instead of '' to listen only
                             # to MCAST_GRP, not all groups on MCAST_PORT
mreq = struct.pack("4sl", socket.inet_aton(MCAST_GRP1), socket.INADDR_ANY)
sock1.setsockopt(socket.IPPROTO_IP, socket.IP_ADD_MEMBERSHIP, mreq)
sock1.setblocking(0)

# Multicast Socket setup - Send 
MCAST_GRP2 = '224.1.1.1' # despite binding to a group we end up accepting all multicast messages in the listen
MCAST_PORT2 = 5007
sock2 = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
sock2.setsockopt(socket.IPPROTO_IP, socket.IP_MULTICAST_TTL, 2)

# Create new threads
thread1 = listenThread(1, "Respond", sock1 ,sock2, 0.05, MCAST_GRP1, MCAST_PORT1, MCAST_GRP2, MCAST_PORT2)

# Start new Threads
thread1.start()

while(1):
    time.sleep(100)


Thread 1 "Respond" Started
Mon Jul  2 02:56:08 2018, Receive, Message: "HeartBeat", Sender: ('172.17.0.7', 44921)
Mon Jul  2 02:56:08 2018, Transmit, Message: "Slave 2 Alive"
Mon Jul  2 02:56:13 2018, Receive, Message: "HeartBeat", Sender: ('172.17.0.7', 44921)
Mon Jul  2 02:56:13 2018, Transmit, Message: "Slave 2 Alive"
Mon Jul  2 02:56:18 2018, Receive, Message: "HeartBeat", Sender: ('172.17.0.7', 44921)
Mon Jul  2 02:56:18 2018, Transmit, Message: "Slave 2 Alive"


KeyboardInterrupt: 